In [1]:
import os
import json
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
# Set seed
np.random.seed(42)

# set font to 12 and times new roman

plt.rcParams.update({'font.size': 12})
plt.rcParams["font.family"] = "Times New Roman"

from utils import *






The `LightGBM` module could not be imported. To enable LightGBM support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `Prophet` module could not be imported. To enable Prophet support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `CatBoost` module could not be imported. To enable CatBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md


In [2]:

os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"

Current working directory: /Users/nikolaushouben/Desktop/WattCast


In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


True

In [4]:
run = wandb.init(project='Wattcast', name='results_synthesis', id = 'results_synthesis', resume=True)

project_name = "Wattcast"
# Initialize your project
api = wandb.Api()
runs = api.runs(project_name)

## Results imports


### Grabbing all metrics from all runs from wandb

In [5]:
name_id_dict = get_run_name_id_dict(runs)

In [6]:
name_id_dict

{'results_in': 'results_synthesis',
 '3_villageinvillage_2': '3_village_village_2_60min',
 '3_villageinvillage_1': '3_village_village_1_60min',
 '3_villageinvillage_0': '3_village_village_0_60min',
 '2_townintown_2': '2_town_town_2_60min',
 '2_townintown_1': '2_town_town_1_60min',
 '2_townintown_0': '2_town_town_0_60min',
 '1_countyinSacramento': '1_county_Sacramento_60min',
 '1_countyinNew_York': '1_county_New_York_60min',
 '1_countyinLos_Angeles': '1_county_Los_Angeles_60min',
 '4_neighborhoodingermany': '4_neighborhood_germany_60min'}

# Paper Figures & Tables

### Mappings for plotting shapes and colors

In [7]:
model2shape = {
    'RandomForest': 'o',
    'LightGBMModel': 's',
    'XGBModel': 'D',
    'BlockRNNModel': 'x',
    'NBEATSModel': '^',
    'TFTModel': 'v',
    'LinearRegressionModel': '*',
    '48-Hour Persistence': 'P'
}


model2color = {
    'RandomForest': 'blue',
    'LightGBMModel': 'orange',
    'XGBModel': 'green',
    'BlockRNNModel': 'red',
    'NBEATSModel': 'purple',
    'TFTModel': 'pink',
    'LinearRegressionModel': 'brown',
    '48-Hour Persistence': 'black'
}


shape2model = {v: k for k, v in model2shape.items()}


metric_dict = {'mape': 'Mean Absolute Percentage Error (MAPE)',
               'rmse': 'Root Mean Squared Error (RMSE)',
                'mae': 'Mean Absolute Error (MAE)',
                'smape': 'Symmetric Mean Absolute Percentage Error (SMAPE)',
                'rmse_skill_score': 'RMSE Skill Score',
                'r2_score': 'R2 Score',
                'max_peak_error': 'Max Peak Error',
                'mean_n_peak_error': 'Mean N Peak Error',
                }

season2color = {'Summer': 'orange', 'Winter': 'blue'}
color2season = {v: k for k, v in season2color.items()}

horizon2color = {'Ground Truth': 'black',
                '1 Hours Ahead': 'blue',
                '4 Hours Ahead': 'green',
                '8 Hours Ahead': 'orange',
                '24 Hours Ahead': 'red',
                '48 Hours Ahead': 'purple'}


model_groups = {'Tree-based': ['RandomForest', 'LightGBMModel', 'XGBModel'],
                         'Neural Network': ['BlockRNNModel', 'NBEATSModel', 'TFTModel'],}

model2group = {}
for group, models in model_groups.items():
    for model in models:
        model2group[model] = group


group2shape = {'Tree-based': 'x', 'Neural Network': 'o'}
shape2group = {v: k for k, v in group2shape.items()}

group2color = {'Tree-based': 'blue', 'Neural Network': 'red'}
color2group = {v: k for k, v in group2color.items()}


scale2unit = {'county': 'GW', 'town': 'MW', 'village': 'kW', 'neighborhood': 'kW'}
unit2kWh = {'W':1/1000, 'kW': 1, 'MW': 1000, 'GW': 1000000}


index2county_locations = {0: 'Los_Angeles', 1: 'New_York', 2: 'Sacramento'}
index2town_locations = {0: 'town_0', 1: 'town_1', 2: 'town_2'}
index2village_locations = {0: 'village_0', 1: 'village_1', 2: 'village_2'}
index2neighborhood_locations = {0: 'neighborhood_0', 1: 'neighborhood_1', 2: 'neighborhood_2'}
scale2ordering = {'county': index2county_locations, 'town': index2town_locations, 'village': index2village_locations, 'neighborhood': index2neighborhood_locations}






In [ ]:
# importing error metric tables (used in most sections below)
key_list = list(name_id_dict.keys())
for key in key_list:
    if key[0] not in ['1', '2', '3', '4', '5']:
        del name_id_dict[key]
        
dfs_sorted = {}
for name, id in name_id_dict.items():
    try:
        run_path = f'wattcast/{project_name}/run-{id}-Errormetrics:latest'
        artifact = run.use_artifact(f'{run_path}', type='run_table')
        artifact_dir = artifact.download()
        with open(os.path.join(artifact_dir, os.listdir(artifact_dir)[0])) as f:
            data = json.load(f)
        df_metrics = pd.DataFrame(data['data'], columns=data['columns'])
        df = df_metrics.sort_values(by=['model', 'season'])
        dfs_sorted[name] = df
    except:
        print(f'Error in {name}')


## Per Model

### Plot 1: Skill Score vs Horizon for each model and season

In [ ]:
# Set these:

spatial_scale = 'town'
metric_of_interest = 'rmse_skill_score'


dfs_sorted_plot = dfs_sorted.copy()
keys = list(dfs_sorted_plot.keys())
for key in keys:
    if not spatial_scale in key:
        del dfs_sorted_plot[key]



fig, axs = plt.subplots(1, len(dfs_sorted_plot), figsize=(5*(len(dfs_sorted_plot)),5), sharex=True, sharey=True)

    
axs = axs.ravel()

# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []

for i, ax in enumerate(axs):
    # Map the colors and shapes to the DataFrame
    name, df = list(dfs_sorted_plot.items())[i]
    df['color'] = df['season'].map(season2color)
    df['shape'] = df['model'].map(model2shape)

    # exclude 24 hour persistence
    df = df.loc[ df['model'] != '48-Hour Persistence']
    
    for shape in set(df['shape']):
        for color in set(df.loc[df['shape'] == shape, 'color']):
            x = df.loc[(df['shape'] == shape) & (df['color'] == color), 'horizon_in_hours']
            y = df.loc[(df['shape'] == shape) & (df['color'] == color), metric_of_interest]
            label = f'{shape2model[shape]} ({color2season[color]})'
            scatter = ax.scatter(x, y, c=color, marker=shape, label=label)
            if label not in legend_labels:
                legend_handles.append(scatter)
                legend_labels.append(label)

    # Set the x ticks and labels
    ax.set_xticks([1, 4, 8, 24, 48])
    ax.set_xlabel('Forecast Horizon (hours)')

    # Set the y label
    if i == 0:
        ax.set_ylabel(f'{metric_dict[metric_of_interest]}')

    # Set the title for each subplot
    ax.set_title(f'{name.split("in")[-1].replace("_", " ")}')

    # Add grid for each subplot
    ax.grid(True)

    # Add a horizontal red dashed line at y = 0
    if metric_of_interest == 'rmse_skill_score':
        ax.axhline(y=0, color='red', linestyle='--')

# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=5, frameon=True)

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()


In [ ]:
for format in ['png', 'pdf']:
    fig.savefig(os.path.join(os.getcwd(),'imgs','figures',f'{spatial_scale}{metric_of_interest}.{format}'), bbox_inches='tight')

### Plot 2: Side by side comparison of models for each season for a selected week

In [ ]:
# set these
run_to_visualize = '4_neighborhoodingermany'
season = 'Summer'
algorithm = 'XGBModel'


scale = run_to_visualize.split('_')[1].split('in')[0]
unit = scale2unit[scale]
conversion = unit2kWh[unit]

In [ ]:
files = get_file_names(project_name, name_id_dict, run_to_visualize, season)

side_by_side_plots_dict= download_plotly_plots(get_latest_plotly_plots(files))

df_all = side_by_side_df(side_by_side_plots_dict)

df_all

### Plot

In [ ]:
# find the day with the highest peak
dt_highest_peak = pd.Timestamp(df_all['Ground Truth'].idxmax()).date()
dt_start = dt_highest_peak - pd.Timedelta(days=2)
dt_end = dt_highest_peak + pd.Timedelta(days=2)

df_all.index = pd.to_datetime(df_all.index)
df_all = df_all.loc[dt_start:dt_end]



fig, ax = plt.subplots(figsize=(12,5))

df_preds = df_all.filter(like=algorithm)
df_preds.columns = [col.split(':')[-1][1:] + ' Ahead' for col in df_preds.columns]
df_gt = df_all.filter(like='Ground')
df_plot = pd.concat([df_gt, df_preds], axis=1)
for col in df_plot.columns:
    ax.plot(df_plot[col], label=col, color=horizon2color[col], linestyle= 'solid' if col == 'Ground Truth' else ':')

ax.set_ylabel(f'Electricity Load ({unit})')
ax.set_title(f'Ground Truth and {algorithm} Predictions for {run_to_visualize}')

ax.grid(True, axis='x')
ax.legend(frameon=True)

In [ ]:
for format in ['png', 'pdf']:
    fig.savefig(os.path.join(os.getcwd(),'imgs','figures',f'plot_3_side-by-side_horizons_heat_wave_{run_to_visualize}_{season}_{algorithm}.{format}'), bbox_inches='tight')

#wandb.log({f'plot_3_side-by-side_horizons_heat_wave_{run_to_visualize}_{season}_{algorithm}': wandb.Image(fig)})

### Plot 3: Comparing Spatial Scales (mean of seasons and datasets)

In [ ]:
metric_of_interest = 'rmse'

df_metrics_all_scales = pd.DataFrame()

for scale, df in dfs_sorted.items():
    scale = scale.split('in')[0]
    df['scale'] = scale
    df_metrics_all_scales = df_metrics_all_scales.append(df)

# marginalize over the season
df_metrics_grouped = df_metrics_all_scales.groupby(['scale', 'model', 'horizon_in_hours']).mean()[[metric_of_interest]].reset_index()

# extract the model with the best performance for each scale and horizon
df_metrics_grouped = df_metrics_grouped.sort_values(by=['scale', metric_of_interest], ascending=False if metric_of_interest == 'rmse_skill_score' else True)
df_metrics_grouped = df_metrics_grouped.drop_duplicates(subset=['scale', 'horizon_in_hours'], keep='first')
df_metrics_grouped = df_metrics_grouped.sort_values(by=['scale', 'horizon_in_hours'])
df_metrics_grouped = df_metrics_grouped.reset_index(drop=True)


# Define colors and shapes based on the season and model
df = df_metrics_grouped
fig, ax = plt.subplots()

# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []


# Map the colors and shapes to the DataFrame
df['shape'] = df['model'].map(model2shape)


for scale, df_scale in df.groupby('scale'): # this for loop is to make sure the order of scales is from county to neighborhood
    for shape in set(df_scale['shape']):
        x = df_scale.loc[(df_scale['shape'] == shape), 'horizon_in_hours']
        y = df_scale.loc[(df_scale['shape'] == shape), 'scale']
        label = f'{shape2model[shape]}'
        scatter = ax.scatter(x, y, marker=shape, label=label, color=model2color[shape2model[shape]])
        if label not in legend_labels:
            legend_handles.append(scatter)
            legend_labels.append(label)

# Set the x ticks and labels
ax.set_xticks([1, 4, 8, 24, 48])
ax.set_xlabel('Forecast Horizon (hours)')


ax.set_ylabel('Spatial Scale')

# Set the title for each subplot
fig.suptitle(f'Best Model by {metric_dict[metric_of_interest]}')

# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.6, -0.25), ncol=2, frameon=True)


# Show the plot
plt.show()


In [ ]:

for format in ['png', 'pdf']:
    fig.savefig(os.path.join(os.getcwd(),'imgs','figures',f'plot_1_overview_across_scales_{metric_of_interest}.{format}'), bbox_inches='tight')

#wandb.log({f'plot_1_overview_across_scales_{metric_of_interest}': fig})

### Plot 4: Comparing Spatial Scales (mean of datasets per season)

In [ ]:
df_metrics_all_scales = pd.DataFrame()

for scale, df in dfs_sorted.items():
    scale = scale.split('in')[0]
    df['scale'] = scale
    df_metrics_all_scales = df_metrics_all_scales.append(df)

df_metrics_grouped = df_metrics_all_scales.groupby(['season', 'scale', 'model', 'horizon_in_hours']).mean()[[metric_of_interest]].reset_index()

season = 'Winter'

fig, axs = plt.subplots(1,2, figsize=(12,5), sharex=True, sharey=True)

axs = axs.ravel()
# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []

for i, ax in enumerate(axs):

    season = list(season2color.keys())[i]

    df_metrics_grouped_season = df_metrics_grouped.loc[df_metrics_grouped['season'] == season]

    df = get_best_model_per_scale_and_horizon(df_metrics_grouped_season, metric_of_interest)

    shape2model = {v: k for k, v in model2shape.items()}

    # Map the colors and shapes to the DataFrame
    df['shape'] = df['model'].map(model2shape)

    for scale, df_scale in df.groupby('scale'):
        for shape in set(df_scale['shape']):
            x = df_scale.loc[(df_scale['shape'] == shape), 'horizon_in_hours']
            y = df_scale.loc[(df_scale['shape'] == shape), 'scale']
            label = f'{shape2model[shape]}'
            scatter = ax.scatter(x, y, marker=shape, label=label, color=model2color[shape2model[shape]])
            if label not in legend_labels:
                legend_handles.append(scatter)
                legend_labels.append(label)

    # Set the x ticks and labels
    ax.set_xticks([1, 4, 8, 24, 48])
    ax.set_xlabel('Forecast Horizon (hours)')
    
    #ax.set_yticklabels(sorted(ax.get_yticklabels(), key=lambda x: x.get_text()))


    # Set the y label
    if i == 0:
        ax.set_ylabel('Spatial Scale')

    # Set the title for each subplot
    ax.set_title(f'{season}')


# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=2, frameon=True)

fig.suptitle(f'Best Model by {metric_dict[metric_of_interest]}')
# Adjust the spacing between subplots
#fig.tight_layout()

# Show the plot
plt.show()


## Tree-based models vs. Deep learning models

### Plot 1

In [ ]:
# Set these:

spatial_scale = 'village'
metric_of_interest = 'rmse_skill_score'

dfs_sorted_plot = dfs_sorted.copy()
keys = list(dfs_sorted_plot.keys())
for key in keys:
    if not spatial_scale in key:
        del dfs_sorted_plot[key]


# Define colors and shapes based on the season and model
color2season = {v: k for k, v in season2color.items()}
shape2model = {v: k for k, v in model2shape.items()}


fig, axs = plt.subplots(1, len(dfs_sorted_plot), figsize=(5*(len(dfs_sorted_plot)),5), sharex=True, sharey=True)
axs = axs.ravel()

# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []

for i, ax in enumerate(axs):
    # Map the colors and shapes to the DataFrame
    name, df = list(dfs_sorted_plot.items())[i]
    # exclude 24 hour persistence
    df = df.loc[df['model'] != '48-Hour Persistence']
    df['group'] = df['model'].map(model2group)
    df = df.groupby(['group', 'horizon_in_hours', 'season']).mean().reset_index()
    df['color'] = df['season'].map(season2color)
    df['shape'] = df['group'].map(group2shape)
    
    
    for shape in set(df['shape']):
        for color in set(df.loc[df['shape'] == shape, 'color']):
            x = df.loc[(df['shape'] == shape) & (df['color'] == color), 'horizon_in_hours']
            y = df.loc[(df['shape'] == shape) & (df['color'] == color), metric_of_interest]
            label = f'{shape2group[shape]} ({color2season[color]})'
            scatter = ax.scatter(x, y, c=color, marker=shape, label=label)
            if label not in legend_labels:
                legend_handles.append(scatter)
                legend_labels.append(label)

    # Set the x ticks and labels
    ax.set_xticks([1, 4, 8, 24, 48])
    ax.set_xlabel('Forecast Horizon (hours)')

    # Set the y label
    if i == 0:
        ax.set_ylabel(f'{metric_dict[metric_of_interest]}')

    # Set the title for each subplot
    ax.set_title(f'{name.split("in")[-1].replace("_", " ")}')

    # Add grid for each subplot
    ax.grid(True)

    # Add a horizontal red dashed line at y = 0
    if metric_of_interest == 'rmse_skill_score':
        ax.axhline(y=0, color='red', linestyle='--')

# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=5, frameon=True)

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()


### Plot 3

In [ ]:
metric_of_interest = 'mape'

df_metrics_all_scales = pd.DataFrame()

for scale, df in dfs_sorted.items():
    scale = scale.split('in')[0]
    df['scale'] = scale
    df_metrics_all_scales = df_metrics_all_scales.append(df)

# marginalize over the season
df_metrics_grouped = df_metrics_all_scales.groupby(['scale', 'model', 'horizon_in_hours']).mean()[[metric_of_interest]].reset_index()

# extract the model with the best performance for each scale and horizon
df_metrics_grouped = df_metrics_grouped.sort_values(by=['scale', metric_of_interest], ascending=False if metric_of_interest == 'rmse_skill_score' else True)
df_metrics_grouped = df_metrics_grouped.drop_duplicates(subset=['scale', 'horizon_in_hours'], keep='first')
df_metrics_grouped = df_metrics_grouped.sort_values(by=['scale', 'horizon_in_hours'])
df_metrics_grouped = df_metrics_grouped.reset_index(drop=True)


# Define colors and shapes based on the season and model
df = df_metrics_grouped
shape2model = {v: k for k, v in model2shape.items()}

fig, ax = plt.subplots()

# Create a list to store the handles and labels for the legend
legend_handles = []
legend_labels = []


# Map the colors and shapes to the DataFrame
df['group'] = df['model'].map(model2group)
df = df.groupby(['group', 'horizon_in_hours', 'scale']).mean().reset_index()

df['shape'] = df['group'].map(group2shape)

for scale, df_scale in df.groupby('scale'): # this for loop is to make sure the order of scales is from county to neighborhood
    for shape in set(df_scale['shape']):
        x = df_scale.loc[(df_scale['shape'] == shape), 'horizon_in_hours']
        y = df_scale.loc[(df_scale['shape'] == shape), 'scale']
        label = f'{shape2group[shape]}'
        scatter = ax.scatter(x, y, marker=shape, label=label, color=group2color[shape2group[shape]])
        if label not in legend_labels:
            legend_handles.append(scatter)
            legend_labels.append(label)

# Set the x ticks and labels
ax.set_xticks([1, 4, 8, 24, 48])
ax.set_xlabel('Forecast Horizon (hours)')

ax.set_ylabel('Spatial Scale')

# Set the title for each subplot
fig.suptitle(f'Best Model by {metric_dict[metric_of_interest]}')

# Create a single legend for all subplots
fig.legend(handles=legend_handles, labels=legend_labels, title='Model', loc='lower center', bbox_to_anchor=(0.6, -0.25), ncol=2, frameon=True)


# Show the plot
plt.show()

for format in ['png', 'pdf']:
    fig.savefig(os.path.join(os.getcwd(),'imgs','figures',f'plot_1_overview_across_scales_{metric_of_interest}.{format}'), bbox_inches='tight')

#wandb.log({f'plot_1_overview_across_scales_{metric_of_interest}': fig})

## MPC Evaluation

In [8]:
name_id_dict

{'results_in': 'results_synthesis',
 '3_villageinvillage_2': '3_village_village_2_60min',
 '3_villageinvillage_1': '3_village_village_1_60min',
 '3_villageinvillage_0': '3_village_village_0_60min',
 '2_townintown_2': '2_town_town_2_60min',
 '2_townintown_1': '2_town_town_1_60min',
 '2_townintown_0': '2_town_town_0_60min',
 '1_countyinSacramento': '1_county_Sacramento_60min',
 '1_countyinNew_York': '1_county_New_York_60min',
 '1_countyinLos_Angeles': '1_county_Los_Angeles_60min',
 '4_neighborhoodingermany': '4_neighborhood_germany_60min'}

In [23]:
# set these
run_to_visualize = list(name_id_dict.keys())[-1]
season = 'Summer'


In [24]:
files = get_file_names(project_name, name_id_dict, run_to_visualize, season)

side_by_side_plots_dict= download_plotly_plots(get_latest_plotly_plots(files))

df_all = side_by_side_df(side_by_side_plots_dict)


In [11]:
df_all = df_all.iloc[:,-3:]

In [22]:
df_all.to_csv('data/results/fc.csv')

In [39]:
df_results = pd.read_csv('data/results/results.csv', index_col=0, parse_dates=True)

px.line(df_results)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('variable=net_load_NBEATSModel ' ... '}<br>value=%{y}<extra></extra>'),
              'legendgroup': 'net_load_NBEATSModel Summer - Horizon: 8 Hours',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'net_load_NBEATSModel Summer - Horizon: 8 Hours',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2012, 6, 16, 2, 0),
                          datetime.datetime(2012, 6, 16, 3, 0),
                          datetime.datetime(2012, 6, 16, 4, 0), ...,
                          datetime.datetime(2012, 7, 10, 23, 0),
                          datetime.datetime(2012, 7, 11, 0, 0),
                          datetime.datetime(2012, 7, 11, 1, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 0.61502433,  0.56904136,  0.09992897, ..., -0.32589625, -0.41794747,
                          -0.34190591]),
              'yaxis': 'y'},
             {'hovertemplate': ('variable=tier1_net_load_NBEATS' ... '}<br>value=%{y}<extra></extra>'),
              'legendgroup': 'tier1_net_load_NBEATSModel Summer - Horizon: 8 Hours',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'tier1_net_load_NBEATSModel Summer - Horizon: 8 Hours',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2012, 6, 16, 2, 0),
                          datetime.datetime(2012, 6, 16, 3, 0),
                          datetime.datetime(2012, 6, 16, 4, 0), ...,
                          datetime.datetime(2012, 7, 10, 23, 0),
                          datetime.datetime(2012, 7, 11, 0, 0),
                          datetime.datetime(2012, 7, 11, 1, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 0.61502433,  0.56904136,  0.09992897, ..., -0.32589625, -0.41794747,
                          -0.34190591]),
              'yaxis': 'y'},
             {'hovertemplate': ('variable=tier2_net_load_NBEATS' ... '}<br>value=%{y}<extra></extra>'),
              'legendgroup': 'tier2_net_load_NBEATSModel Summer - Horizon: 8 Hours',
              'line': {'color': '#00cc96', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'tier2_net_load_NBEATSModel Summer - Horizon: 8 Hours',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2012, 6, 16, 2, 0),
                          datetime.datetime(2012, 6, 16, 3, 0),
                          datetime.datetime(2012, 6, 16, 4, 0), ...,
                          datetime.datetime(2012, 7, 10, 23, 0),
                          datetime.datetime(2012, 7, 11, 0, 0),
                          datetime.datetime(2012, 7, 11, 1, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([0., 0., 0., ..., 0., 0., 0.]),
              'yaxis': 'y'},
             {'hovertemplate': ('variable=bss_p_ch_NBEATSModel ' ... '}<br>value=%{y}<extra></extra>'),
              'legendgroup': 'bss_p_ch_NBEATSModel Summer - Horizon: 8 Hours',
              'line': {'color': '#ab63fa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'bss_p_ch_NBEATSModel Summer - Horizon: 8 Hours',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2012, 6, 16, 2, 0),
                          datetime.datetime(2012, 6, 16, 3, 0),
                          datetime.datetime(2012, 6, 16, 4, 0), ...,
                          datetime.datetime(2012, 7, 10, 23, 0),
                          datetime.datetime(2012, 7, 11, 0, 0),
                          datetime.datetime(2012, 7, 11, 1, 0)], dtype=ob

In [40]:
df_results.filter(like='en')

,bss_en_NBEATSModel Summer - Horizon: 8 Hours,bss_en_LinearRegressionModel Summer - Horizon: 8 Hours,bss_en_Ground Truth
2012-06-16 02:00:00,1.5,1.50000,1.500000
2012-06-16 03:00:00,2.0,1.67291,1.759944
2012-06-16 04:00:00,2.0,2.00000,2.000000
2012-06-16 05:00:00,2.0,2.00000,2.000000
2012-06-16 06:00:00,1.5,1.50000,1.500000
...,...,...,...
2012-07-10 21:00:00,2.0,2.00000,1.769726
2012-07-10 22:00:00,1.5,1.50000,1.269726
2012-07-10 23:00:00,1.0,1.00000,1.000000
2012-07-11 00:00:00,0.5,0.50000,0.500000


In [41]:
pd.read_csv('data/results/costs.csv')

,Unnamed: 0,horizon,tier1,tier2,peak,total
0,NBEATSModel Summer - Horizon: 8 Hours,8.0,106.565153,6.076497,13.492226,126.133876
1,LinearRegressionModel Summer - Horizon: 8 Hours,8.0,106.450161,6.416857,13.492226,126.359245
2,Ground Truth,8.0,107.897171,4.410383,11.361972,123.669526
